In [1]:
import string
import scipy.io as sc
import os
import cv2
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.


In [7]:
data_train = sc.loadmat("./dataset/IIIT5K/trainCharBound.mat")
data_test = sc.loadmat("./dataset/IIIT5K/testCharBound.mat")

In [8]:
# data['trainCharBound'][0][0]

In [9]:
# path = './dataset/IIIT5K/train/'
# for root, direc, files in os.walk(path):
#         for file in files:
#             img = Image.open(os.path.join(path+file))
#             img = img.resize((128,32), Image.ANTIALIAS)
#             img.save(os.path.join(path+file)) 



In [10]:
discription = {}

for i in data_train['trainCharBound'][0]:
    discription[(i[0][0]).split('/')[1]] = i[1][0]
for i in data_test['testCharBound'][0]:
    discription[(i[0][0]).split('/')[1]] = i[1][0]

In [11]:
len(discription)

5000

In [12]:
char_list = string.ascii_letters+string.digits
def encode_to_labels(txt):
    # encoding each output word into digits
    dig_lst = []
    for index, char in enumerate(txt):
        try:
            dig_lst.append(char_list.index(char))
        except:
            print(char)
       
    return dig_lst

In [13]:

def create_tensor(path,max_label_len=0):
    img_tensor = []
    txt_tensor = []
    input_length = []
    label_length = []
    orig_txt = []
    i=0
    for root, direc, files in os.walk(path):
#         print(len(files))
        for file in files:
#             if file.endswith('.png'):
#                 print(os.path.join(path+file))
#             if file not in discription.keys():
#                 continue
            i=i+1
            img = cv2.imread(os.path.join(path+file))
            img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
            w, h = img.shape
            if h != 128 or w != 32:
                print('errr')
            
            img = np.expand_dims(img , axis = 2)

            img = img/255.0

            txt = discription[file]

            if len(txt) > max_label_len:
                max_label_len = len(txt)
            
            orig_txt.append(txt)   
            label_length.append(len(txt))
            input_length.append(31)
            img_tensor.append(img)
            txt_tensor.append(encode_to_labels(txt))
    print(i)
    return orig_txt, label_length, input_length, img_tensor, txt_tensor,max_label_len
            
    

In [14]:
path_train = './dataset/IIIT5K/train/'

orig_txt, train_label_length, train_input_length, training_img, training_txt, max_label_len = create_tensor(path_train)


5000


In [15]:
max_label_len

22

In [16]:
for i,arr in enumerate(training_txt):
    training_txt[i] = np.array( training_txt[i])
    training_txt[i] = training_txt[i].flatten()

In [17]:
training_txt

[array([41, 43, 34, 47, 26, 45, 30]),
 array([41, 26, 43, 36, 34, 39, 32]),
 array([44,  0, 11, 20, 19,  4, 18]),
 array([50, 14, 20]),
 array([29, 40, 37, 28, 30]),
 array([32, 26, 27, 27, 26, 39, 26]),
 array([43, 30, 32, 30, 39, 28, 50]),
 array([44, 45, 26, 45, 30]),
 array([27, 26, 39, 36]),
 array([40, 31]),
 array([34, 39, 29, 34, 26]),
 array([43,  4, 18,  2, 20,  4]),
 array([12,  8, 18, 18,  8, 14, 13]),
 array([36, 34, 39, 32, 31, 34, 44, 33, 30, 43]),
 array([28, 37, 30, 26, 43]),
 array([28, 33, 26, 39, 39, 30, 37]),
 array([46, 39, 34, 47, 30, 43, 44, 26, 37]),
 array([44, 45, 46, 29, 34, 40, 44]),
 array([33, 40, 37, 37, 50, 48, 40, 40, 29]),
 array([28, 40, 38]),
 array([33, 14, 12,  4]),
 array([37, 14,  0, 13]),
 array([19,  7,  0, 19]),
 array([33, 40, 38, 30]),
 array([37, 40, 26, 39, 44]),
 array([27, 40, 43, 29, 30, 43]),
 array([29, 40, 29, 26]),
 array([27, 26, 45, 40, 45, 30]),
 array([35, 26, 38, 38, 46]),
 array([56, 53, 36, 38]),
 array([60, 55, 36, 38]),
 a

In [18]:
train_padded_txt = pad_sequences(training_txt, maxlen=max_label_len, padding='post', value = len(char_list))

In [19]:
import keras
import keras.backend as K
from keras.layers import *
from keras import Model

In [20]:
# input with shape of height=32 and width=128 
inputs = Input(shape=(32,128,1))
 
# convolution layer with kernel size (3,3)
conv_1 = Conv2D(64, (3,3), activation = 'relu', padding='same')(inputs)
# poolig layer with kernel size (2,2)
pool_1 = MaxPool2D(pool_size=(2, 2), strides=2)(conv_1)
 
conv_2 = Conv2D(128, (3,3), activation = 'relu', padding='same')(pool_1)
pool_2 = MaxPool2D(pool_size=(2, 2), strides=2)(conv_2)
 
conv_3 = Conv2D(256, (3,3), activation = 'relu', padding='same')(pool_2)
 
conv_4 = Conv2D(256, (3,3), activation = 'relu', padding='same')(conv_3)
# poolig layer with kernel size (2,1)
pool_4 = MaxPool2D(pool_size=(2, 1))(conv_4)
 
conv_5 = Conv2D(512, (3,3), activation = 'relu', padding='same')(pool_4)
# Batch normalization layer
batch_norm_5 = BatchNormalization()(conv_5)
 
conv_6 = Conv2D(512, (3,3), activation = 'relu', padding='same')(batch_norm_5)
batch_norm_6 = BatchNormalization()(conv_6)
pool_6 = MaxPool2D(pool_size=(2, 1))(batch_norm_6)
 
conv_7 = Conv2D(512, (2,2), activation = 'relu')(pool_6)
 
squeezed = Lambda(lambda x: K.squeeze(x, 1))(conv_7)
 
# bidirectional LSTM layers with units=128
blstm_1 = Bidirectional(LSTM(128, return_sequences=True, dropout = 0.2))(squeezed)
blstm_2 = Bidirectional(LSTM(128, return_sequences=True, dropout = 0.2))(blstm_1)
 
outputs = Dense(len(char_list)+1, activation = 'softmax')(blstm_2)
 
act_model = Model(inputs, outputs)

In [21]:
act_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 32, 128, 1)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 32, 128, 64)       640       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 16, 64, 64)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 16, 64, 128)       73856     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 8, 32, 128)        0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 8, 32, 256)        295168    
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 8, 32, 256)        590080    
__________

In [22]:

labels = Input(name='the_labels', shape=[max_label_len], dtype='float32')
input_length = Input(name='input_length', shape=[1], dtype='int64')
label_length = Input(name='label_length', shape=[1], dtype='int64')
 
def ctc_lambda_func(args):
    y_pred, labels, input_length, label_length = args
    return K.ctc_batch_cost(labels, y_pred, input_length, label_length)
 
loss_out = Lambda(ctc_lambda_func, output_shape=(1,), name='ctc')([outputs, labels, input_length, label_length])
model = Model(inputs=[inputs, labels, input_length, label_length], outputs=loss_out)

W0326 12:35:38.409437 26292 deprecation.py:323] From C:\Users\asus\Anaconda3\envs\ML_GPU\lib\site-packages\keras\backend\tensorflow_backend.py:4249: to_int32 (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.cast` instead.
W0326 12:35:38.853145 26292 deprecation.py:323] From C:\Users\asus\Anaconda3\envs\ML_GPU\lib\site-packages\tensorflow\python\ops\array_ops.py:1354: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0326 12:35:39.267653 26292 deprecation.py:323] From C:\Users\asus\Anaconda3\envs\ML_GPU\lib\site-packages\keras\backend\tensorflow_backend.py:4229: to_int64 (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.cast` instead.


In [23]:
from keras.callbacks import ModelCheckpoint
model.compile(loss={'ctc': lambda y_true, y_pred: y_pred}, optimizer = 'adam')
 
filepath="best_model.hdf5"
checkpoint = ModelCheckpoint(filepath=filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='auto')
callbacks_list = [checkpoint]

W0326 12:35:39.328543 26292 deprecation_wrapper.py:119] From C:\Users\asus\Anaconda3\envs\ML_GPU\lib\site-packages\keras\optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



In [24]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 32, 128, 1)   0                                            
__________________________________________________________________________________________________
conv2d_8 (Conv2D)               (None, 32, 128, 64)  640         input_2[0][0]                    
__________________________________________________________________________________________________
max_pooling2d_5 (MaxPooling2D)  (None, 16, 64, 64)   0           conv2d_8[0][0]                   
__________________________________________________________________________________________________
conv2d_9 (Conv2D)               (None, 16, 64, 128)  73856       max_pooling2d_5[0][0]            
__________________________________________________________________________________________________
max_poolin

In [20]:
training_img = np.array(training_img)
train_input_length = np.array(train_input_length)
train_label_length = np.array(train_label_length)
 
model.fit(x=[training_img, train_padded_txt, train_input_length, train_label_length], y=np.zeros(5000), batch_size=264, epochs = 100, validation_split=0.1, callbacks = callbacks_list)

Train on 4500 samples, validate on 500 samples
Epoch 1/100
4500/4500 [==============================] - 23s 5ms/step - loss: 30.3583 - val_loss: 21.6817

Epoch 00001: val_loss improved from inf to 21.68175, saving model to best_model.hdf5
Epoch 2/100
4500/4500 [==============================] - 13s 3ms/step - loss: 21.2048 - val_loss: 21.4524

Epoch 00002: val_loss improved from 21.68175 to 21.45235, saving model to best_model.hdf5
Epoch 3/100
4500/4500 [==============================] - 13s 3ms/step - loss: 21.0596 - val_loss: 21.2870

Epoch 00003: val_loss improved from 21.45235 to 21.28702, saving model to best_model.hdf5
Epoch 4/100
4500/4500 [==============================] - 13s 3ms/step - loss: 20.8488 - val_loss: 21.2823

Epoch 00004: val_loss improved from 21.28702 to 21.28228, saving model to best_model.hdf5
Epoch 5/100
4500/4500 [==============================] - 13s 3ms/step - loss: 20.5149 - val_loss: 20.7951

Epoch 00005: val_loss improved from 21.28228 to 20.79514, savin

KeyboardInterrupt: 

In [45]:
path = './dataset/test/'
for root, direc, files in os.walk(path):
        for file in files:
            img = Image.open(os.path.join(path+file))
            img = img.resize((128,32), Image.ANTIALIAS)
            img.save(os.path.join(path+file)) 


test_imgs = [] 
label = []
print("original text = ")
print('\n')
for root, direc, files in os.walk(path):
    for file in files:
        i=i+1
        img = cv2.imread(os.path.join(path+file))
        img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        w, h = img.shape
        if h != 128 or w != 32:
            print('errr')

        img = np.expand_dims(img , axis = 2)
        
        img = img/255.0
        test_imgs.append(img)
        
        print(file)
        label.append(file)

act_model.load_weights('best_model.hdf5')
test_imgs = np.array(test_imgs)
prediction = act_model.predict(test_imgs)

out = K.get_value(K.ctc_decode(prediction, input_length=np.ones(prediction.shape[0])*prediction.shape[1],
                         greedy=True)[0][0])
i=0
print('\n')
print("predicted text = ")
print('\n')
for x in out:
    

    for p in x:  
        if int(p) != -1:
            print(char_list[int(p)], end = '')
            
    print('\n')
    i+=1

original text = 


aphasia.jpg
clock.jpg
draught.jpg
ephemeral.jpg
hammer.jpg
knife.jpg
light.jpg
paper.jpg
pencil.jpg
silver.jpg
spoon.jpg


predicted text = 


AhAiA

clock

Drght

EPHEMERAL

Bawgner

Kanife

Iight

Paper

penci

STlver

SPoon

